In [1]:
from data_harvester import DataHarvester
from datetime import datetime
import pytz
import os
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)

# Time Zone
TZ = pytz.timezone('Australia/Sydney')

# KsqlDB Address
QUERY_URL = 'https://api.dev.unimelb-traffico.cloud.edu.au/query'
# TABLE_NAME = 'DELAY_CONGESTION_TAGGED'
TABLE_NAME = 'ED_5MINHW8'

# Personal Cert
KEY_ROOT = os.path.join('..', 'key')
CA_CERT = os.path.join(KEY_ROOT, 'ca.crt')
CLIENT_CERT = os.path.join(KEY_ROOT, 'client.crt')
CLIENT_KEY = os.path.join(KEY_ROOT, 'client.key')

In [2]:
# Research Configuration
LINK_ID = [2727, 2726]
START_TIME = datetime(year=2021, month=9, day=10, hour=8, minute=0, second=0)
END_TIME = datetime(year=2021, month=9, day=11, hour=11, minute=0, second=0)

data_harvester = DataHarvester(ca_cert=CA_CERT, client_crt=CLIENT_CERT, client_key=CLIENT_KEY,
                               query_url=QUERY_URL, table_name=TABLE_NAME)

data_df = data_harvester.get_batch_df(link_ids=LINK_ID, start_time=START_TIME, end_time=END_TIME)
# data_df['WINDOWSTART'] = pd.to_datetime(data_df['WINDOWSTART'], unit='s')
# data_window = data_df['LATEST_ED'].rolling(window=3)
# j = 0
# for i in data_window:
#     print("============")
#     print(i)
#     j += 1
#     if j == 4:
#         break
print(len(data_df))
data_df.head(10)

2021-09-16 23:11:11,453 - INFO - Downloading raw data from KsqlDB...


12780


,ID,LATEST_ED,ENOUGHDATA,TimeStamp
0,2727.0,12.0,false,1631224800
1,2726.0,-54.0,false,1631224800
2,2727.0,12.0,false,1631224830
3,2726.0,-56.0,false,1631224830
4,2727.0,13.0,false,1631224860
5,2726.0,-57.0,false,1631224860
6,2727.0,13.0,false,1631224890
7,2726.0,-58.0,false,1631224890
8,2727.0,13.0,false,1631224920
9,2726.0,-59.0,false,1631224920


In [3]:
interval = (int(END_TIME.timestamp()) - int(START_TIME.timestamp()))
time_lines = [int(START_TIME.timestamp()) + i * 30 for i in range(interval//30)]
time_lines = pd.Series(time_lines, name='TimeStamp')

In [4]:
print(len(time_lines))
print(len(data_df))
new_data_df = data_df.merge(time_lines, how='right', left_on='WINDOWSTART', right_on='TimeStamp')
print(len(new_data_df))
new_data_df

3240
12780


KeyError: 'WINDOWSTART'